In [1]:
import pandas as pd
import gmaps
import gmaps.datasets
import requests
import os
# Import the API key.
from config import g_key
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
CityDataToLoad = os.path.join("data","WeatherPy_vacation.csv")
city_data_df = pd.read_csv(CityDataToLoad)
city_data_df = city_data_df.set_index("City_ID")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches),Date
City_ID,,,,,,,,,,,,
0,Bonfim,BR,3.08,-59.95,87.37,61,96,4.90,light rain,0.0,0.0,2020-07-09 20:21:58
1,Victoria,HK,22.29,114.16,86.00,74,100,11.01,overcast clouds,0.0,0.0,2020-07-09 20:21:58
3,Manicore,BR,-5.81,-61.30,89.60,55,20,5.82,few clouds,0.0,0.0,2020-07-09 20:21:58
6,Aloleng,PH,16.13,119.78,82.74,80,99,7.11,light rain,0.0,0.0,2020-07-09 20:21:59
7,Iracoubo,GF,5.48,-53.20,81.82,78,35,6.20,light rain,0.0,0.0,2020-07-09 20:21:59


In [3]:
PotentialVacationSpots = ["Bose", "Florence", "Cicciano", "Melfi" ]

In [4]:
CityA_df = city_data_df.loc[(city_data_df["City"] == PotentialVacationSpots[0])]
CityB_df = city_data_df.loc[(city_data_df["City"] == PotentialVacationSpots[1])]
CityC_df = city_data_df.loc[(city_data_df["City"] == PotentialVacationSpots[2])]
CityD_df = city_data_df.loc[(city_data_df["City"] == PotentialVacationSpots[3])]

In [5]:
CityA_LatLng = (CityA_df["Lat"].to_numpy()[0],CityA_df["Lng"].to_numpy()[0])
CityB_LatLng = (CityB_df["Lat"].to_numpy()[0],CityB_df["Lng"].to_numpy()[0])
CityC_LatLng = (CityC_df["Lat"].to_numpy()[0],CityC_df["Lng"].to_numpy()[0])
CityD_LatLng = (CityD_df["Lat"].to_numpy()[0],CityD_df["Lng"].to_numpy()[0])

In [6]:
fig = gmaps.figure()
TravelPath = gmaps.directions_layer(CityA_LatLng, CityD_LatLng, waypoints = [CityB_LatLng, CityC_LatLng], travel_mode = "DRIVING")
fig.add_layer(TravelPath)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
TravelCitiesDF = pd.concat([CityA_df, CityB_df, CityC_df, CityD_df], axis = 0)

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = TravelCitiesDF[["City", "Country", "Max Temp","Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [9]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [11]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [14]:
# Map the Vacation spots with a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))